In [17]:
# Pytorch sin regularizacion ni optimizacion aplicado al dataset de wine usado en one vs all:

import numpy as np
import torch
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


In [3]:
# Carga desde CSV
data = np.loadtxt("wine_preparado.csv", delimiter=',')
X, y = data[:, 1:], data[:, 0]

# Dividimos los datos para entrenamiento y test
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y)

In [5]:
# Convertimos a tensores
X_train_t = torch.from_numpy(X_train).float().cuda()
y_train_t = torch.from_numpy(y_train - 1).long().cuda()  # etiquetas 0,1,2 = 1,2,3
X_test_t = torch.from_numpy(X_test).float().cuda()
y_test_t = torch.from_numpy(y_test - 1).long().cuda()

# Aplicamos el Dataloader para que el batch sea liviano
train_ds = TensorDataset(X_train_t, y_train_t)
train_loader = DataLoader(train_ds, batch_size=32, shuffle=True)

In [7]:
# Armamos el modelo custom:

import torch.nn as nn
import torch.nn.functional as F

class ModelCustom2(nn.Module):
    def __init__(self, D_in, H, D_out):
        super().__init__()
        self.fc1 = nn.Linear(D_in, H)
        self.bn1 = nn.BatchNorm1d(H)
        self.fc2 = nn.Linear(H, H)
        self.bn2 = nn.BatchNorm1d(H)
        self.fc3 = nn.Linear(H, D_out)
    def forward(self, x):
        x = F.relu(self.bn1(self.fc1(x)))
        x = F.relu(self.bn2(self.fc2(x)))
        x = self.fc3(x)
        return x

model = ModelCustom2(13, 64, 3).cuda()  # 13 etiquetas, 64 ocultas, 3 clases de salida

In [9]:
#Preparamos el entrenamiento:

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.1, momentum=0.9)

epochs = 100
log_each = 10
losses = []

model.train()
for e in range(1, epochs + 1):
    for xb, yb in train_loader:
        y_pred = model(xb)
        loss = criterion(y_pred, yb)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    losses.append(loss.item())
    if not e % log_each:
        print(f"Epoch {e}/{epochs} Loss {np.mean(losses):.5f}")

Epoch 10/100 Loss 0.65251
Epoch 20/100 Loss 0.68040
Epoch 30/100 Loss 0.63634
Epoch 40/100 Loss 0.70494
Epoch 50/100 Loss 0.69314
Epoch 60/100 Loss 0.70630
Epoch 70/100 Loss 0.68338
Epoch 80/100 Loss 0.67723
Epoch 90/100 Loss 0.67108
Epoch 100/100 Loss 0.67211


In [10]:
model.eval()
with torch.no_grad():
    y_logits = model(X_test_t)
    y_pred = torch.argmax(y_logits, dim=1)

acc = accuracy_score(y_test_t.cpu().numpy(), y_pred.cpu().numpy())
print("Accuracy:", acc)

Accuracy: 0.7222222222222222


In [14]:
# Mostrar las predicciones:
pred_labels = (y_pred + 1).cpu().numpy()  # vuelve al formato 1,2,3
print("Predicciones del conjunto de test:")
print(pred_labels)

# Para comparar real con prediccion:
true_labels = (y_test_t + 1).cpu().numpy()
for i, (p, t) in enumerate(zip(pred_labels, true_labels)):
    print(f"Ejemplo: {i:03d} -> Prediccion: {p}, Real: {t}")

Predicciones del conjunto de test:
[1 3 1 3 3 1 1 2 2 1 2 3 1 2 1 2 2 2 3 1 2 3 1 1 3 2 1 3 3 3 1 2 2 3 3 3]
Ejemplo: 000 -> Prediccion: 1, Real: 1
Ejemplo: 001 -> Prediccion: 3, Real: 3
Ejemplo: 002 -> Prediccion: 1, Real: 1
Ejemplo: 003 -> Prediccion: 3, Real: 2
Ejemplo: 004 -> Prediccion: 3, Real: 2
Ejemplo: 005 -> Prediccion: 1, Real: 1
Ejemplo: 006 -> Prediccion: 1, Real: 1
Ejemplo: 007 -> Prediccion: 2, Real: 2
Ejemplo: 008 -> Prediccion: 2, Real: 2
Ejemplo: 009 -> Prediccion: 1, Real: 3
Ejemplo: 010 -> Prediccion: 2, Real: 2
Ejemplo: 011 -> Prediccion: 3, Real: 3
Ejemplo: 012 -> Prediccion: 1, Real: 1
Ejemplo: 013 -> Prediccion: 2, Real: 3
Ejemplo: 014 -> Prediccion: 1, Real: 1
Ejemplo: 015 -> Prediccion: 2, Real: 2
Ejemplo: 016 -> Prediccion: 2, Real: 2
Ejemplo: 017 -> Prediccion: 2, Real: 1
Ejemplo: 018 -> Prediccion: 3, Real: 2
Ejemplo: 019 -> Prediccion: 1, Real: 1
Ejemplo: 020 -> Prediccion: 2, Real: 2
Ejemplo: 021 -> Prediccion: 3, Real: 2
Ejemplo: 022 -> Prediccion: 1, Re